In [3]:
# kafka_producer.py
from kafka import KafkaProducer
import json
from time import sleep
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

producer = KafkaProducer(
    bootstrap_servers=['13.58.11.37:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

df = pd.read_csv("data/netflix_titles.csv")

# Function to train a simple recommendation model
def train_recommendation_model(data):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'].fillna(''))

    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    return cosine_sim

recommendation_model = train_recommendation_model(df)

with open('data/netflix_titles.csv', encoding='utf-8') as f:
    next(f)  # Skip the header row
    for line in f:
        # Convert each CSV line to a dictionary
        data = dict(zip(df.columns, line.strip().split(',')))

        title_index = df.index[df['title'] == data['title']]
        if not title_index.empty:
            title_index = title_index[0]
            data['recommendations'] = [df['title'][i] for i in recommendation_model[title_index].argsort()[::-1][1:6]]
        else:
            data['recommendations'] = [] 

        producer.send('netflix-data', value=data)
        sleep(0.1)

KeyError: 'title'